# LexNLPを用いた法律テキストからの情報抽出

このノートブックでは、[LexNLP](https://lexpredict-lexnlp.readthedocs.io/en/latest/)を使って、法律テキストから情報抽出をする方法を紹介します。LexNLPは、オープンソースの情報抽出のためのツールです。

## 準備

### パッケージのインストール

パッケージをインストールしたら、ランタイムを再起動しましょう。

In [2]:
!pip install -q lexnlp==1.8.0 textract==1.6.4
!pip install -qU pandas nltk

     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 61 kB 382 kB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 14.5 MB 14.9 MB/s 
     |████████████████████████████████| 10.1 MB 47.8 MB/s 
     |████████████████████████████████| 24.2 MB 1.6 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 660 kB 40.4 MB/s 
     |████████████████████████████████| 238 kB 50.7 MB/s 
     |████████████████████████████████| 352 kB 56.4 MB/s 
     |████████████████████████████████| 294 kB 49.3 MB/s 
     |████████████████████████████████| 106 kB 57.9 MB/s 
     |████████████████████████████████| 111 kB 56.0 MB/s 
     |████████████████████████████████| 69 kB 6.6 MB/s 
     |████████████████████████████████| 5.6 MB 56.6 MB/s 
     |████████████████████████████████| 10.1 MB 10.8 MB/s 
     |████████████████████████████████| 32.8 MB 111 kB/s 
     |████████████

### インポート

モジュールをインポートします。
`lexnlp.extract`モジュールには、非構造化テキストから構造化されたデータを抽出するためのメソッドが含まれています。日付や金額などさまざまな種類のデータを抽出できるので、このあとやってみましょう。`lexnlp.nlp`モジュールには、単語分割やステミング、見出し語化など、自然言語処理タスクをサポートするメソッドが含まれています。



- [lexnlp.extract: Extracting structured data from unstructured text](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/extract.html)
- [lexnlp.nlp: Natural language processing](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/nlp/nlp.html)



In [3]:
from pprint import pprint

import lexnlp.extract.en.acts
import lexnlp.extract.en.amounts
import lexnlp.extract.en.money
import lexnlp.extract.en.dates
import lexnlp.extract.en.ratios
import lexnlp.extract.en.definitions
import lexnlp.extract.en.copyright
import lexnlp.extract.en.durations
import lexnlp.nlp.en.tokens
import lexnlp.nlp.en.segments.sections
import lexnlp.nlp.en.transforms.tokens
import nltk
import pandas as pd
import textract
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### データのアップロード

本章のフォルダ内にある`test - Copy.docs`をアップロードします。

In [4]:
from google.colab import files
test_upload = files.upload()

Saving test - Copy.docx to test - Copy.docx


`.docs`を文字列に変換しておきましょう。

In [5]:
text = textract.process("test - Copy.docx").decode("utf-8")
type(text)

str

## 法律名（Acts）の抽出

`lexnlp.extract.en.acts`モジュールを用いて、テキストから法律名（Act）を抽出できます。法律名の例としては以下が挙げられます。

- Advancing Hope Act
- AGOA Acceleration Act of 2004
- section 12 of the Agricultural Act of 1954
- sections 751(a)(1) and 777(i)(1) of the Act

In [17]:
data_contract = list(lexnlp.extract.en.acts.get_acts(text))
pprint(data_contract)

df = pd.DataFrame(data=data_contract, columns=data_contract[0].keys())
df["Act_annotations"] = list(lexnlp.extract.en.acts.get_acts_annotations(text))
print(df.head(10))

[{'act_name': 'Securities Exchange Act',
  'ambiguous': False,
  'location_end': 6264,
  'location_start': 6233,
  'section': '',
  'value': 'Securities Exchange Act of 1934',
  'year': '1934'},
 {'act_name': 'Securities Exchange Act',
  'ambiguous': False,
  'location_end': 6377,
  'location_start': 6346,
  'section': '',
  'value': 'Securities Exchange Act of 1934',
  'year': '1934'},
 {'act_name': 'Securities Act',
  'ambiguous': False,
  'location_end': 9176,
  'location_start': 9158,
  'section': '',
  'value': 'Securities Act.\n\n“',
  'year': ''},
 {'act_name': 'Securities Act',
  'ambiguous': False,
  'location_end': 15419,
  'location_start': 15403,
  'section': '',
  'value': 'Securities Act, ',
  'year': ''},
 {'act_name': 'Securities Act',
  'ambiguous': False,
  'location_end': 15707,
  'location_start': 15691,
  'section': '',
  'value': 'Securities Act, ',
  'year': ''},
 {'act_name': 'Securities Act',
  'ambiguous': False,
  'location_end': 15821,
  'location_start': 15

## 量（Amounts）の抽出

`lexnlp.extract.en.amounts`モジュールを使うことで、テキストから量（Amount）に関係する情報を抽出できます。たとえば、以下が挙げられます。

- THIRTY-SIX THOUSAND TWO-HUNDRED SIXTY-SIX AND 2/100
- total 2 million people
- total 2.035 billion tons of
- fifteen cats
- 20,000,000
- thirty-five million units
- 10K files
- 1 trill

In [18]:
amts = list(lexnlp.extract.en.amounts.get_amounts(text))
locs = list(lexnlp.extract.en.amounts.get_amount_annotations(text))
pprint(amts)

data_amts = list(zip(amts, locs))
df = pd.DataFrame(data=data_amts, columns=["Amount", "Location in Contract"])
print(df.head())

[Decimal('1933.0'),
 Decimal('1000000.0'),
 Decimal('19.0'),
 Decimal('2.0'),
 Decimal('2022.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('100000.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('2.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('2.0'),
 Decimal('13.0'),
 Decimal('14.0'),
 Decimal('1934.0'),
 Decimal('3.0'),
 Decimal('1934.0'),
 Decimal('50.0'),
 Decimal('1.0'),
 Decimal('3.0'),
 Decimal('3.0'),
 Decimal('1.0'),
 Decimal('4.0'),
 Decimal('501.0'),
 Decimal('5.0'),
 Decimal('1.0'),
 Decimal('50.0'),
 Decimal('48.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('1.0'),
 Decimal('304.0'),
 Decimal('305.0'),
 Decimal('306.0'),
 Decimal('354.0'),
 Decimal('368.0'),
 Decimal('1036.0'),
 Decimal('1202.0'),
 Decimal('1986.0')]
      Amount              Location in Contract
0     1933.0   [amount] at (113..119), loc: en
1  1000000.0   [amount] at (604..612), loc: en
2       19.0   [amount] at (648..651), loc: en
3        2.0  

## 日付（Dates）の抽出

`lexnlp.extract.en.dates`モジュールを使うと、日付を抽出できます。以下は日付の例です。

- February 1, 1998
- 2017-06-01
- 1st day of June, 2017
- 31 October 2016
- 15th of March 2000

In [19]:
data_dates = list(lexnlp.extract.en.dates.get_dates(text))
df = pd.DataFrame(data=data_dates, columns=["Date"])
print(df.head())

         Date
0  2022-02-19


## 金額（Money）の抽出

`lexnlp.extract.en.money`モジュールを使うと、金額を抽出できます。以下は金額の例です。

 five dollars
- 5 dollars
- 5 USD
- $5

In [22]:
data_money = list(lexnlp.extract.en.money.get_money(text))
df = pd.DataFrame(data=data_money, columns=["Amount", "Currency"])
print(df.head())

      Amount Currency
0  1000000.0      USD
1   100000.0      USD
2        2.0      USD


## 比率（Ratios）の抽出

`lexnlp.extract.en.ratios`モジュールを使うことで、比率を抽出できます。以下は比率の例です。

- 3:1
- 3.0:1.0
- three to one


In [23]:
data_ratios = list(lexnlp.extract.en.ratios.get_ratios(text))
df = pd.DataFrame(data=data_ratios, columns=["Ratios"])
print(df.head())
print("No ratios in the contract")

Empty DataFrame
Columns: [Ratios]
Index: []
No ratios in the contract


## 定義（Definitions）の抽出

`lexnlp.extract.en.definitions`モジュールを使うことで、テキストから定義を抽出できます。以下の文における`X`が定義の例として挙げられます。

- X shall [not] include …
- X shall have the meaning …
- X is hereby changed to …
- X shall be interpreted …
- X shall for purposes …
- X shall be deemed to …
- X shall refer to …
- X shall mean …
- X is defined …
- The word “X” includes every description of …
- The term “X” means …
- Description of term (the “x”)

In [24]:
data_acts = list(lexnlp.extract.en.definitions.get_definitions(text))
df = pd.DataFrame(data=data_acts, columns=["Acts"])
print(df.head(20))

                       Acts
0            SECURITIES ACT
1                  Investor
2           Purchase Amount
3         Conversion Amount
4           Cash-Out Amount
5             Capital Stock
6         Change of Control
7     Converting Securities
8         Dissolution Event
9           Dividend Amount
10         Equity Financing
11  Initial Public Offering
12          Liquidity Event
13          Liquidity Price
14                  Options
15                 Proceeds
16         Promised Options
17                     Safe
18      SafePreferred Stock
19               Safe Price


## コピーライト（Copyrights）の抽出

`lexnlp.extract.en.copyright`モジュールを使うことでコピーライトを抽出できます。

In [25]:
data_cp_ann = list(lexnlp.extract.en.copyright.get_copyright_annotations(text))

data_cp = list(lexnlp.extract.en.copyright.get_copyright(text))

df = pd.DataFrame(
    data=list(zip(data_cp, data_cp_ann)),
    columns=["CopyRight Annotations", "CopyRight data"],
)
print(df.head())

            CopyRight Annotations                   CopyRight data
0  ((c), None, Dissolution Event)  Dissolution Event, (3862, 3883)


## 期間（Durations）の抽出

`lexnlp.extract.en.durations`モジュールを使うことで、期間を抽出できます。


In [26]:
data_dur = list(lexnlp.extract.en.durations.get_durations(text))
data_dur_ann = list(lexnlp.extract.en.durations.get_duration_annotations_list(text))

df = pd.DataFrame(
    data=list(zip(data_dur, data_dur_ann)), columns=["Durations", "Duration Annotation"]
)
print(df.head())

           Durations                     Duration Annotation
0  (hour, 48.0, 2.0)   [duration] at (17772..17781), loc: en


## トークンの抽出

`get_nouns`や`get_verbs`などのメソッドを使うことで、テキストから名詞や動詞を抽出できます。また、ステミングや見出し語化などの処理もできます。

In [27]:
print("Noun")
print(list(lexnlp.nlp.en.tokens.get_nouns(text))[:10])

print("Stemming")
print(
    lexnlp.nlp.en.tokens.get_stem_list(
        text, stemmer=nltk.stem.lancaster.LancasterStemmer()
    )[:10]
)

print("Lematization")
print(list(lexnlp.nlp.en.tokens.get_verbs(text, lemmatize=True))[:10])

Noun
['THIS', 'INSTRUMENT', 'AND', 'ANY', 'SECURITIES', 'ISSUABLE', 'PURSUANT', 'HERETO', 'HAVE', 'NOT']
Stemming
['thi', 'instru', 'and', 'any', 'sec', 'issu', 'pursu', 'hereto', 'hav', 'not']
Lematization
['s', 'describe', 'be', 'agree', 'have', 'modify', 'fill', 'bracket', 'be', 'define']


## セクションの抽出

`lexnlp.nlp.en.segments.sections`モジュールを使うことで、テキストからセクションを抽出できます。

In [28]:
pprint(list(lexnlp.nlp.en.segments.sections.get_sections(text)))

[]


## Bigramの分布

In [29]:
pprint(dict(lexnlp.nlp.en.transforms.tokens.get_bigram_distribution(text)))

{('$', '100000.'): 1,
 ('$', '1000000'): 1,
 ('%', 'of'): 2,
 ('(', '1'): 1,
 ('(', '2'): 1,
 ('(', 'A'): 3,
 ('(', 'B'): 3,
 ('(', 'C'): 1,
 ('(', 'Signature'): 1,
 ('(', 'Simple'): 1,
 ('(', 'THE'): 1,
 ('(', 'a'): 5,
 ('(', 'and'): 1,
 ('(', 'as'): 1,
 ('(', 'b'): 5,
 ('(', 'but'): 1,
 ('(', 'c'): 4,
 ('(', 'd'): 7,
 ('(', 'e'): 3,
 ('(', 'even'): 1,
 ('(', 'except'): 1,
 ('(', 'excluding'): 1,
 ('(', 'f'): 1,
 ('(', 'g'): 1,
 ('(', 'i'): 15,
 ('(', 'ii'): 16,
 ('(', 'iii'): 7,
 ('(', 'in'): 1,
 ('(', 'including'): 3,
 ('(', 'or'): 2,
 ('(', 's'): 2,
 ('(', 'subject'): 1,
 ('(', 'that'): 1,
 ('(', 'the'): 4,
 ('(', 'to'): 1,
 ('(', 'treating'): 1,
 ('(', 'within'): 1,
 ('(', 'without'): 3,
 ('(', 'x'): 1,
 ('(', 'y'): 1,
 (')', ')'): 2,
 (')', ','): 7,
 (')', '.'): 6,
 (')', '.If'): 1,
 (')', ':'): 3,
 (')', ';'): 2,
 (')', 'All'): 1,
 (')', 'Any'): 2,
 (')', 'Dissolution'): 1,
 (')', 'Equity'): 1,
 (')', 'IN'): 1,
 (')', 'In'): 1,
 (')', 'Junior'): 1,
 (')', 'Liquidation'): 1,
 (')